In [562]:
import json
import pandas as pd
import geopandas as gpd
import os
import statsmodels.api as sma
import altair as alt

In [710]:
# assign directory
directory = 'data/raw/gunbroker/'

In [711]:
data_list = []

In [712]:
# iterate over files in directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        file = open(f)
        data = json.load(file)
        data_list = data_list + data.get('results')

In [713]:
len(data_list)

37514

In [714]:
df = pd.DataFrame.from_records(data_list)

In [715]:
df.columns

Index(['fflID', 'name', 'company', 'address1', 'address2', 'city', 'state',
       'zip', 'phone', 'fax', 'cellPhone', 'otherPhone', 'hours', 'longGunFee',
       'longGunDescription', 'handGunFee', 'handGunDescription', 'nicsFee',
       'nicsDescription', 'otherFee', 'otherDescription', 'promotionalText',
       'website', 'latitude', 'longitude', 'licenseNumber', 'distance',
       'licenseOnFile', 'links'],
      dtype='object')

In [716]:
df.drop(['links','distance'], axis=1, inplace=True)

In [717]:
df.drop_duplicates(inplace=True)

In [718]:
df.head()

,fflID,name,company,address1,address2,city,state,zip,phone,fax,...,nicsFee,nicsDescription,otherFee,otherDescription,promotionalText,website,latitude,longitude,licenseNumber,licenseOnFile
0,57794,Craig Sirna,Tactical Assault Specialist,19009 RAVENNA RD,,Chagrin Falls,OH,44023,4408340696,,...,0.0,,0.0,,,NaN,41.35040,-81.22385,4-34-XXX-XX-XX-07720,True
1,39791,,A&Z Sales and Service,302 west 7th St,,Evart,MI,49631,231-734-5070,,...,0.0,,0.0,,,NaN,43.90047,-85.26265,4-38-XXX-XX-XX-08968,True
2,63430,"James E. Arens, Jr.","Arens Ballistics Company, Ltd.",1035 Gartner Dr.,,Obetz,OH,43207,614-321-1699,,...,5.0,,5.0,,,NaN,39.87400,-82.97070,4-31-XXX-XX-XX-08547,True
3,39281,Dean Williams,"Williams, Dean Dennis",2564 N. Aragon Ave,,Kettering,OH,45420,937-902-3731,,...,20.0,,0.0,,,NaN,39.71350,-84.11720,4-31-XXX-XX-XX-04249,True
4,60812,Zac Hendrix,Vance Outdoors,4250 Alum Creek Drive,,Obetz,OH,43207,6144895025,,...,0.0,,0.0,,,NaN,39.88289,-82.93063,4-31-XXX-XX-XX-06052,True


In [719]:
len(df)

29399

In [720]:
df.to_csv("data/processed/gunbroker.csv", index=False)

### Match with FFL

In [721]:
ffl = pd.read_csv("data/processed/dealers-list/2023/0123-ffl-list-annotated.csv", dtype={"lic_regn":str,"lic_dist":str,"lic_seqn":str})

In [855]:
len(ffl)

77336

In [856]:
ffl["lic_type"].unique()

array([1, 7, 8, 2])

In [722]:
ffl["licenseNumber"] = ffl["lic_regn"] + "-" + ffl["lic_dist"].str[1:3] + "-XXX-XX-XX-" + ffl["lic_seqn"]

In [723]:
ffl.sample()

,lic_regn,lic_dist,lic_cnty,lic_type,lic_xprdte,lic_seqn,license_name,business_name,premise_street,premise_city,...,tract_latino,tract_median_income,business,pawn,sport,defense,big_box,small_business,rdi,licenseNumber
17149,1,058,223,1,3F,12950,SERGEANT'S ARMS LLC,NaN,228 WOODRIDGE DRIVE,DOUGLASVILLE,...,0.00859,74968.0,True,False,False,False,False,False,Residential,1-58-XXX-XX-XX-12950


In [724]:
ffl_merge = ffl.merge(df, on="licenseNumber", how="outer")

In [725]:
len(ffl_merge)

79025

In [726]:
ffl_merge["big_box"] = ffl_merge["big_box"].fillna(False)
ffl_merge["pawn"] = ffl_merge["pawn"].fillna(False)

In [727]:
ffl_merge_trim = ffl_merge.loc[~pd.isna(ffl_merge["fflID"]) | (ffl_merge["big_box"] == True) | (ffl_merge["pawn"] == True)]

In [728]:
ffl_merge_trim

,lic_regn,lic_dist,lic_cnty,lic_type,lic_xprdte,lic_seqn,license_name,business_name,premise_street,premise_city,...,handGunDescription,nicsFee,nicsDescription,otherFee,otherDescription,promotionalText,website,latitude_y,longitude_y,licenseOnFile
3,6,004,13.0,7.0,4D,12422,GUN VALLEY ARMS LLC,NaN,81 RAMAH CIRCLE SOUTH SUITE 5,AGAWAM,...,,0.0,,0.0,,,NaN,42.071110,-72.623440,True
12,6,004,27.0,1.0,4F,14926,"JJT ENTERPRISES, LLC",DOWN RANGE SPORTS,590 SUMMER STREET,BARRE,...,,0.0,,0.0,,,NaN,42.412800,-72.097740,True
14,6,004,15.0,1.0,3E,36592,"EVERETT, DOUGLAS FORDE",SWIFT RIVER GUNWORKS,450 STATE ST,BELCHERTOWN,...,,0.0,,0.0,,,NaN,42.265540,-72.441470,False
21,6,004,15.0,7.0,3D,14383,KC SMALL ARMS LLC,NaN,412 MAIN STREET,EASTHAMPTON,...,NICS and MIRCS registration,0.0,,0.0,,,NaN,42.253770,-72.694860,True
24,6,004,13.0,1.0,3G,12049,"YACOVONE, STEPHEN ALAN",INSIGHT SALES,143E SHAKER RD SUITE 200E,EAST LONGMEADOW,...,,0.0,,0.0,,,NaN,42.066140,-72.510060,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,0.0,,0.0,,,NaN,35.530020,-81.277290,False
79021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,0.0,,0.0,,,NaN,36.262240,-79.224370,False
79022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,0.0,,0.0,,,NaN,34.942620,-80.754100,False
79023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,25.0,,25.0,,,NaN,36.243460,-90.961530,False


In [729]:
ffl_merge_trim[pd.isna(ffl_merge_trim["latitude_x"])]["latitude_x"] = ffl_merge_trim[pd.isna(ffl_merge_trim["latitude_x"])]["latitude_y"]

/var/folders/1x/hkrq_4px1_x314p6_8yb37m80000gn/T/ipykernel_59684/3182209145.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ffl_merge_trim[pd.isna(ffl_merge_trim["latitude_x"])]["latitude_x"] = ffl_merge_trim[pd.isna(ffl_merge_trim["latitude_x"])]["latitude_y"]


In [730]:
ffl_merge_trim[pd.isna(ffl_merge_trim["longitude_x"])]["longitude_x"] = ffl_merge_trim[pd.isna(ffl_merge_trim["longitude_x"])]["longitude_y"]

/var/folders/1x/hkrq_4px1_x314p6_8yb37m80000gn/T/ipykernel_59684/82958175.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ffl_merge_trim[pd.isna(ffl_merge_trim["longitude_x"])]["longitude_x"] = ffl_merge_trim[pd.isna(ffl_merge_trim["longitude_x"])]["longitude_y"]


In [731]:
ffl_merge_trim.columns

Index(['lic_regn', 'lic_dist', 'lic_cnty', 'lic_type', 'lic_xprdte',
       'lic_seqn', 'license_name', 'business_name', 'premise_street',
       'premise_city', 'premise_state', 'premise_zip_code', 'mail_street',
       'mail_city', 'mail_state', 'mail_zip_code', 'voice_phone', 'id',
       'file_year', 'file_month', 'license_name_clean', 'original_address',
       'longitude_x', 'latitude_x', 'clean_address', 'statefp', 'countyfp',
       'county', 'county_area', 'state_x', 'tractce', 'tract_area',
       'county_population', 'county_white', 'county_black', 'county_asian',
       'county_latino', 'county_median_income', 'county_poverty',
       'tract_population', 'tract_white', 'tract_black', 'tract_asian',
       'tract_latino', 'tract_median_income', 'business', 'pawn', 'sport',
       'defense', 'big_box', 'small_business', 'rdi', 'licenseNumber', 'fflID',
       'name', 'company', 'address1', 'address2', 'city', 'state_y', 'zip',
       'phone', 'fax', 'cellPhone', 'otherPhone',

In [732]:
ffl_merge_trim.drop(['county_population', 'county_white', 'county_black', 'county_asian', 'county_latino', 
                     'county_median_income', 'county_poverty', 'tractce', 'tract_area', 'tract_population', 
                     'tract_white', 'tract_black', 'tract_asian', 'tract_latino', 'tract_median_income', 
                     'business', 'pawn', 'sport', 'defense', 'big_box', 'small_business', 'rdi', 
                     'latitude_y', 'longitude_y'], axis=1, inplace=True)

/var/folders/1x/hkrq_4px1_x314p6_8yb37m80000gn/T/ipykernel_59684/2212552323.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ffl_merge_trim.drop(['county_population', 'county_white', 'county_black', 'county_asian', 'county_latino',


In [733]:
ffl_merge_trim.rename(columns={'longitude_x':'longitude', 'latitude_x':'latitude'}, inplace=True)

/var/folders/1x/hkrq_4px1_x314p6_8yb37m80000gn/T/ipykernel_59684/206733380.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ffl_merge_trim.rename(columns={'longitude_x':'longitude', 'latitude_x':'latitude'}, inplace=True)


### Assign a county

In [734]:
counties = gpd.read_file("data/processed/counties.geojson")

In [735]:
counties

,statefp,countyfp,county,county_area,state,geoid,geometry
0,02,013,Aleutians East,15009.939752,Alaska,02013,"MULTIPOLYGON (((-3801432.512 3133472.767, -380..."
1,02,016,Aleutians West,14116.419570,Alaska,02016,"MULTIPOLYGON (((-4900590.329 3834004.986, -490..."
2,28,107,Panola,705.131639,Mississippi,28107,"MULTIPOLYGON (((513070.490 -662207.945, 513069..."
3,28,101,Newton,579.601941,Mississippi,28101,"MULTIPOLYGON (((614290.086 -847983.437, 614317..."
4,28,027,Coahoma,583.152412,Mississippi,28027,"MULTIPOLYGON (((474830.268 -692448.051, 474470..."
...,...,...,...,...,...,...,...
3216,37,077,Granville,536.498459,North Carolina,37077,"MULTIPOLYGON (((1455833.489 -317345.762, 14557..."
3217,37,111,McDowell,445.994701,North Carolina,37111,"MULTIPOLYGON (((1207528.929 -416252.481, 12075..."
3218,27,021,Cass,2413.993603,Minnesota,27021,"MULTIPOLYGON (((96243.198 814680.164, 96242.80..."
3219,27,057,Hubbard,999.559340,Minnesota,27057,"MULTIPOLYGON (((59375.994 845151.916, 59373.36..."


In [736]:
# Set to NAD 1983 Albers North America - https://epsg.io/102008
counties = counties.to_crs("EPSG:4269")

In [737]:
# Create geodataframe
gdf = gpd.GeoDataFrame(ffl_merge_trim, geometry=gpd.points_from_xy(ffl_merge_trim.longitude, ffl_merge_trim.latitude))

In [738]:
# Set to NAD 1983 Albers North America - https://epsg.io/102008
gdf = gdf.set_crs("EPSG:4269")

In [739]:
# Merge with county
gdf_county = gdf.sjoin(counties, how="left", predicate='intersects')
gdf_county.drop('index_right', axis=1, inplace=True)

In [740]:
gdf_county.head()

,lic_regn,lic_dist,lic_cnty,lic_type,lic_xprdte,lic_seqn,license_name,business_name,premise_street,premise_city,...,promotionalText,website,licenseOnFile,geometry,statefp_right,countyfp_right,county_right,county_area_right,state,geoid
3,6,004,13.0,7.0,4D,12422,GUN VALLEY ARMS LLC,NaN,81 RAMAH CIRCLE SOUTH SUITE 5,AGAWAM,...,,NaN,True,POINT (-72.63132 42.08435),25,013,Hampden,634.251128,Massachusetts,25013
12,6,004,27.0,1.0,4F,14926,"JJT ENTERPRISES, LLC",DOWN RANGE SPORTS,590 SUMMER STREET,BARRE,...,,NaN,True,POINT (-72.09790 42.40992),25,027,Worcester,1579.193920,Massachusetts,25027
14,6,004,15.0,1.0,3E,36592,"EVERETT, DOUGLAS FORDE",SWIFT RIVER GUNWORKS,450 STATE ST,BELCHERTOWN,...,,NaN,False,POINT (-72.43958 42.26582),25,015,Hampshire,545.209270,Massachusetts,25015
21,6,004,15.0,7.0,3D,14383,KC SMALL ARMS LLC,NaN,412 MAIN STREET,EASTHAMPTON,...,,NaN,True,POINT (-72.69247 42.25332),25,015,Hampshire,545.209270,Massachusetts,25015
24,6,004,13.0,1.0,3G,12049,"YACOVONE, STEPHEN ALAN",INSIGHT SALES,143E SHAKER RD SUITE 200E,EAST LONGMEADOW,...,,NaN,True,POINT (-72.51679 42.05884),25,013,Hampden,634.251128,Massachusetts,25013


In [741]:
gdf_county = gdf_county.sort_values(["company","state"])

In [742]:
gdf_county.to_file("data/processed/gunbroker_locations.geojson", driver='GeoJSON')

### Group by county

In [743]:
gdf_county_merge = gdf_county.groupby(["geoid"]).size().reset_index(name="dealers")

In [744]:
gdf_county_merge.head()

,geoid,dealers
0,01001,12
1,01003,35
2,01005,5
3,01007,1
4,01009,11


In [770]:
counties_merge_full = counties[["geoid","state","county","county_area"]].merge(gdf_county_merge, on="geoid", how="left")

In [771]:
counties_merge_full.head()

,geoid,state,county,county_area,dealers
0,02013,Alaska,Aleutians East,15009.939752,NaN
1,02016,Alaska,Aleutians West,14116.419570,1.0
2,28107,Mississippi,Panola,705.131639,8.0
3,28101,Mississippi,Newton,579.601941,6.0
4,28027,Mississippi,Coahoma,583.152412,3.0


### Merge with census data

In [772]:
# Load and clean county demographic data
# county_race_df = pd.read_csv("data/processed/census/acs5_2020_race_counties.csv", dtype={"geoid": str})
# county_race_df["white_pct"] = county_race_df["white_alone"] / county_race_df["universe"]
# county_race_df["black_pct"] = county_race_df["black_alone"] / county_race_df["universe"]
# county_race_df["latino_pct"] = county_race_df["latino_alone"] / county_race_df["universe"]
# county_race_df = county_race_df.rename(columns={"universe":"population","white_alone":"white","black_alone":"black","latino_alone":"latino"})
# county_race_df = county_race_df[["geoid","population","white","black","latino","white_pct","black_pct","latino_pct"]]

In [773]:
# # Load and clean county income data
# county_income_df = pd.read_csv("data/processed/census/acs5_"+str(year)+"_medianhouseholdincome_counties.csv", dtype={"geoid": str})
# county_income_df_clean = clean_income(county_income_df, "county")

In [774]:
# Load and clean county poverty data
county_poverty_df = pd.read_csv("data/processed/census/acs5_2020_poverty_counties.csv", dtype={"geoid": str})
county_poverty_df["poverty_pct"] = county_poverty_df["income_past12months_below_poverty_level"] / county_poverty_df["universe"]
county_poverty_df = county_poverty_df.rename(columns={"universe":"population"})
county_poverty_df = county_poverty_df[["geoid","population","poverty_pct"]]

In [775]:
# Merge
# final_df = final_df.merge(county_race_df, on="geoid", how="left")
# df = df.merge(county_income_df_clean, on="geoid", how="left")
final_df = counties_merge_full.merge(county_poverty_df, on="geoid", how="left")

### Create dealer rate

In [776]:
final_df["dealer_rate"] = final_df["dealers"] / final_df["county_area"]

### Add neighbor dealer rate

In [777]:
neighbor_file = open("neighbor-counties.json")
neighbors = json.load(neighbor_file)

In [778]:
neighbor_dealer_sum = []
neighbor_area_sum = []
# neighbor_area_population = []

for row in final_df.itertuples():
    neighbor_filter = final_df[final_df["geoid"].isin(neighbors.get(row.geoid))]
    neighbor_dealer_sum.append(neighbor_filter["dealers"].sum())
    neighbor_area_sum.append(neighbor_filter["county_area"].sum())
    # neighbor_area_population.append(neighbor_filter["population"].sum())

final_df["neighbor_dealers"] = neighbor_dealer_sum
final_df["neighbor_area"] = neighbor_area_sum
# final_df["neighbor_population"] = neighbor_area_population

In [779]:
final_df["neighbor_dealers_rate"] = (final_df["dealers"] + final_df["neighbor_dealers"]) / (final_df["county_area"] + final_df["neighbor_area"])

In [780]:
final_df['neighbor_dealers_rate_adj'] = final_df.apply(lambda x: x["dealer_rate"] if x["county_area"] >= 50 else x["neighbor_dealers_rate"], axis = 1)

In [781]:
final_df.sort_values("neighbor_dealers_rate_adj", ascending=False).head(10)

,geoid,state,county,county_area,dealers,population,poverty_pct,dealer_rate,neighbor_dealers,neighbor_area,neighbor_dealers_rate,neighbor_dealers_rate_adj
1223,48439,Texas,Tarrant,902.304892,273.0,2050487.0,0.113635,0.302558,485.0,5380.480972,0.120647,0.302558
216,48201,Texas,Harris,1777.482550,397.0,4634207.0,0.156065,0.223350,331.0,7009.966841,0.082845,0.223350
10,48113,Texas,Dallas,908.613868,177.0,2592698.0,0.145804,0.194802,657.0,4649.510694,0.150051,0.194802
628,22055,Louisiana,Lafayette,269.208664,48.0,238082.0,0.166850,0.178300,56.0,4985.524363,0.019792,0.178300
2264,12103,Florida,Pinellas,608.126655,107.0,955568.0,0.115877,0.175950,156.0,2134.181766,0.095905,0.175950
1257,40143,Oklahoma,Tulsa,587.018072,99.0,640621.0,0.143420,0.168649,101.0,6297.340636,0.029051,0.168649
2711,48085,Texas,Collin,886.103501,148.0,1000193.0,0.062556,0.167023,395.0,4770.391728,0.095996,0.167023
128,42045,Pennsylvania,Delaware,190.603645,31.0,544692.0,0.099096,0.162641,129.0,2220.316862,0.066365,0.162641
828,13067,Georgia,Cobb,344.517747,56.0,744737.0,0.086042,0.162546,136.0,1954.210882,0.083524,0.162546
1079,49035,Utah,Salt Lake,807.368563,130.0,1130965.0,0.085993,0.161017,182.0,13763.079168,0.021413,0.161017


### Merge with mortality data

In [782]:
cdc = pd.read_csv("data/processed/cdc-data/mult2021_counties.csv", dtype={"geoid":str})

In [783]:
cdc.drop(['statefp', 'state'], axis=1, inplace=True)

In [784]:
cdc.columns

Index(['state_abbr', 'geoid', 'type_manner', 'American Indian', 'Asian',
       'Black', 'Multiple Races', 'Pacific Islander', 'White',
       'county_population', 'county_white', 'county_black', 'county_asian',
       'county_aian', 'county_latino', 'county_median_income', 'white_rate',
       'black_rate', 'asian_rate', 'aian_rate', 'total_rate'],
      dtype='object')

In [785]:
firearm_homicide = cdc.loc[cdc["type_manner"] == "firearm-homicide"]
firearm_suicide = cdc.loc[cdc["type_manner"] == "firearm-suicide"]
non_firearm_homicide = cdc.loc[cdc["type_manner"] == "non-firearm-homicide"]

In [786]:
firearm_homicide = final_df.merge(firearm_homicide, on="geoid", how="left")
firearm_suicide = final_df.merge(firearm_suicide, on="geoid", how="left")
non_firearm_homicide = final_df.merge(non_firearm_homicide, on="geoid", how="left")

In [787]:
firearm_homicide = firearm_homicide.fillna(0)
firearm_suicide = firearm_suicide.fillna(0)
non_firearm_homicide = non_firearm_homicide.fillna(0)

In [867]:
top_counties = firearm_homicide[firearm_homicide["population"] > 50000].sort_values("total_rate", ascending=False).head(10)

In [868]:
top_counties

,geoid,state,county,county_area,dealers,population,poverty_pct,dealer_rate,neighbor_dealers,neighbor_area,...,county_black,county_asian,county_aian,county_latino,county_median_income,white_rate,black_rate,asian_rate,aian_rate,total_rate
74,29510,Missouri,St. Louis,66.033597,7.0,296577.0,0.204325,0.106007,104.0,1937.612384,...,138482.0,10292.0,623.0,12530.0,45782.0,11.951983,163.198105,9.716284,0.000000,80.404583
362,28049,Mississippi,Hinds,877.291927,22.0,226933.0,0.203985,0.025077,75.0,4971.823091,...,171125.0,1765.0,249.0,3561.0,45380.0,24.615817,89.408327,0.000000,0.000000,70.881649
2679,22071,Louisiana,Orleans,350.215864,7.0,377648.0,0.230310,0.019988,68.0,6514.895194,...,229253.0,11186.0,492.0,21452.0,43258.0,17.510068,90.729456,17.879492,0.000000,59.041684
443,24510,Maryland,Baltimore,92.051215,7.0,580311.0,0.200337,0.076045,54.0,1269.556064,...,370749.0,15016.0,1575.0,32627.0,52164.0,14.597031,81.726451,0.000000,0.000000,54.460262
2353,51760,Virginia,Richmond,62.461669,2.0,219038.0,0.208982,0.032020,46.0,681.861794,...,104961.0,4741.0,277.0,16220.0,51421.0,13.771186,88.604339,21.092596,0.000000,46.677398
1242,05069,Arkansas,Jefferson,913.775764,12.0,61785.0,0.207769,0.013132,92.0,4448.117252,...,37989.0,1029.0,199.0,1482.0,40402.0,15.330369,60.543842,97.181730,0.000000,41.102124
2920,22017,Louisiana,Caddo,936.823519,36.0,238498.0,0.228698,0.038428,104.0,5919.503494,...,119245.0,3301.0,754.0,6942.0,42003.0,6.493808,76.313472,30.293850,0.000000,40.700041
2252,01101,Alabama,Montgomery,799.911830,35.0,218646.0,0.187627,0.043755,40.0,4508.772594,...,130779.0,6949.0,311.0,7997.0,51963.0,9.315076,58.877954,14.390560,0.000000,37.535714
2910,47157,Tennessee,Shelby,785.018637,64.0,919250.0,0.189679,0.081527,63.0,3942.518502,...,502392.0,25424.0,985.0,60266.0,52092.0,12.425146,60.510518,3.933291,101.522843,37.048465
1673,13215,Georgia,Muscogee,221.006487,16.0,188353.0,0.200432,0.072396,37.0,2381.924378,...,89196.0,5108.0,581.0,15127.0,47418.0,11.572437,69.509843,0.000000,0.000000,36.332375


In [869]:
top_counties["county_state"] = top_counties["county"] + ", " + top_counties["state"] 

In [875]:
alt.Chart(top_counties).mark_bar().encode(
    x=alt.X('total_rate:Q', title=""),
    y=alt.Y('county_state:N', sort='-x', title=""),
).properties(
    width=250,
    height=200,
    title="Gun homicides per 100,000"
)

alt.Chart(...)

### Trim and bin the data

In [790]:
firearm_homicide_large = firearm_homicide[firearm_homicide['population'] >= 50000]
firearm_suicide_large = firearm_suicide[firearm_suicide['population'] >= 50000]
non_firearm_homicide_large = non_firearm_homicide[non_firearm_homicide['population'] >= 50000]

In [791]:
# bins = [0, 0.10, 0.30, 1]
# labels = [1,2,3]
# final_df_large['black_pct_qcut'] = pd.cut(final_df_large['black_pct'], bins=bins, labels=labels)

In [799]:
firearm_homicide_large['poverty_pct_qcut'] = pd.qcut(firearm_homicide_large['poverty_pct'], 4, labels=["1", "2", "3", "4"])

/var/folders/1x/hkrq_4px1_x314p6_8yb37m80000gn/T/ipykernel_59684/2853979747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  firearm_homicide_large['poverty_pct_qcut'] = pd.qcut(firearm_homicide_large['poverty_pct'], 4, labels=["1", "2", "3", "4"])


In [800]:
firearm_homicide_large.head()

,geoid,state,county,county_area,dealers,population,poverty_pct,dealer_rate,neighbor_dealers,neighbor_area,...,county_asian,county_aian,county_latino,county_median_income,white_rate,black_rate,asian_rate,aian_rate,total_rate,poverty_pct_qcut
7,51510,Virginia,Alexandria,15.467289,4.0,156746.0,0.094210,0.258610,73.0,999.490621,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,1
10,48113,Texas,Dallas,908.613868,177.0,2592698.0,0.145804,0.194802,657.0,4649.510694,...,170142.0,5152.0,1055049.0,61870.0,16.275977,26.836965,3.526466,0.0,10.905067,3
16,48231,Texas,Hunt,882.023044,27.0,93531.0,0.142701,0.030611,240.0,4930.505330,...,1259.0,875.0,16332.0,57467.0,1.464751,0.000000,0.000000,0.0,1.039479,3
18,48427,Texas,Starr,1229.104495,7.0,63423.0,0.341753,0.005695,66.0,4720.769749,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,4
21,48451,Texas,Tom Green,1540.545538,14.0,112438.0,0.121560,0.009088,8.0,8342.872504,...,1517.0,211.0,48092.0,57670.0,9.638245,21.440823,0.000000,0.0,5.899954,2


In [801]:
firearm_homicide_large.columns

Index(['geoid', 'state', 'county', 'county_area', 'dealers', 'population',
       'poverty_pct', 'dealer_rate', 'neighbor_dealers', 'neighbor_area',
       'neighbor_dealers_rate', 'neighbor_dealers_rate_adj', 'state_abbr',
       'type_manner', 'American Indian', 'Asian', 'Black', 'Multiple Races',
       'Pacific Islander', 'White', 'county_population', 'county_white',
       'county_black', 'county_asian', 'county_aian', 'county_latino',
       'county_median_income', 'white_rate', 'black_rate', 'asian_rate',
       'aian_rate', 'total_rate', 'poverty_pct_qcut'],
      dtype='object')

### Run regression

In [809]:
firearm_homicide[firearm_homicide['county_black'] >= 5000]

,geoid,state,county,county_area,dealers,population,poverty_pct,dealer_rate,neighbor_dealers,neighbor_area,...,county_black,county_asian,county_aian,county_latino,county_median_income,white_rate,black_rate,asian_rate,aian_rate,total_rate
2,28107,Mississippi,Panola,705.131639,8.0,33679.0,0.196413,0.011345,22.0,3124.469128,...,16346.0,283.0,130.0,667.0,37232.0,6.267628,30.588523,0.0,0.0,17.606150
3,28101,Mississippi,Newton,579.601941,6.0,20560.0,0.229912,0.010352,45.0,5257.932965,...,6519.0,108.0,1125.0,417.0,42176.0,7.865964,15.339776,0.0,0.0,9.427292
4,28027,Mississippi,Coahoma,583.152412,3.0,22055.0,0.365178,0.005144,17.0,3879.389933,...,17404.0,109.0,75.0,357.0,30761.0,0.000000,51.712250,0.0,0.0,39.673793
5,22065,Louisiana,Madison,650.899620,2.0,9146.0,0.360704,0.003073,20.0,2902.129138,...,7089.0,0.0,7.0,250.0,32585.0,0.000000,56.425448,0.0,0.0,35.916315
6,51540,Virginia,Charlottesville,10.258863,1.0,44552.0,0.230966,0.097477,6.0,726.117463,...,8716.0,3318.0,122.0,2623.0,59598.0,3.228618,11.473153,0.0,0.0,4.235763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3207,37133,North Carolina,Onslow,905.912986,44.0,177251.0,0.124067,0.048570,47.0,3568.320132,...,27094.0,4209.0,683.0,25483.0,51560.0,3.062764,14.763416,0.0,0.0,4.032726
3208,37033,North Carolina,Caswell,428.251884,8.0,21371.0,0.161574,0.018681,97.0,3664.706262,...,7385.0,226.0,56.0,958.0,47938.0,0.000000,13.540961,0.0,0.0,4.421062
3212,37157,North Carolina,Rockingham,572.705326,30.0,89366.0,0.179308,0.052383,157.0,3751.498501,...,16474.0,655.0,314.0,5630.0,45697.0,7.623811,12.140342,0.0,0.0,7.687999
3213,27037,Minnesota,Dakota,586.866817,35.0,422344.0,0.057950,0.059639,101.0,3455.960720,...,26711.0,21032.0,825.0,31069.0,88468.0,0.608189,3.743776,0.0,0.0,1.175721


In [818]:
# making the scatter plot on latitude and longitude
fig = alt.Chart(firearm_homicide_large[firearm_homicide_large['county_black'] >= 10000]).mark_point().encode(x='neighbor_dealers_rate_adj',y='black_rate')
# making the regression line using transform_regression function and add with the scatter plot
final_plot = fig + fig.transform_regression('neighbor_dealers_rate_adj','black_rate').mark_line()
final_plot.encode(
    tooltip=['state', 'county', 'county_population']
).facet(
    'poverty_pct_qcut:N',
    columns = 4
)

alt.FacetChart(...)

In [817]:
# making the scatter plot on latitude and longitude
fig = alt.Chart(firearm_homicide_large[firearm_homicide_large['county_white'] >= 5000]).mark_point().encode(x='neighbor_dealers_rate_adj',y='white_rate')
# making the regression line using transform_regression function and add with the scatter plot
final_plot = fig + fig.transform_regression('neighbor_dealers_rate_adj','white_rate').mark_line()
final_plot.encode(
    tooltip=['state', 'county', 'county_population']
).facet(
    'poverty_pct_qcut:N',
    columns = 4
)

alt.FacetChart(...)

In [861]:
# making the scatter plot on latitude and longitude
fig = alt.Chart(firearm_homicide_large[firearm_homicide_large['county_black'] >= 5000]).mark_point().encode(x='neighbor_dealers_rate_adj',y='black_rate')
# making the regression line using transform_regression function and add with the scatter plot
final_plot = fig + fig.transform_regression('neighbor_dealers_rate_adj','black_rate').mark_line()
final_plot.encode(
    tooltip=['state', 'county', 'county_population']
).properties(
    width=200,
    height=200
).facet(
    'state:N',
    columns = 6
)

alt.FacetChart(...)

In [804]:
def multiple(x_data, y_data):
    X = x_data
    y = y_data
    X2 = sma.add_constant(X)
    est = sma.OLS(y, X2)
    est2 = est.fit()
    print(est2.summary())

In [822]:
trim = firearm_homicide_large[firearm_homicide_large['county_black'] >= 5000]
multiple(trim[['neighbor_dealers_rate_adj','poverty_pct']], trim['black_rate'])

                            OLS Regression Results                            
Dep. Variable:             black_rate   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                     86.43
Date:                Tue, 21 Mar 2023   Prob (F-statistic):           2.25e-33
Time:                        11:42:24   Log-Likelihood:                -2408.1
No. Observations:                 554   AIC:                             4822.
Df Residuals:                     551   BIC:                             4835.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [813]:
trim = firearm_homicide_large[firearm_homicide_large['county_white'] >= 5000]
multiple(trim[['neighbor_dealers_rate_adj','poverty_pct']], trim['white_rate'])

                            OLS Regression Results                            
Dep. Variable:             white_rate   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     70.93
Date:                Tue, 21 Mar 2023   Prob (F-statistic):           3.49e-29
Time:                        11:22:09   Log-Likelihood:                -2642.8
No. Observations:                 840   AIC:                             5292.
Df Residuals:                     837   BIC:                             5306.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [814]:
firearm_homicide_large[['black_rate',"neighbor_dealers_rate_adj"]].corr()

,black_rate,neighbor_dealers_rate_adj
black_rate,1.000000,0.075203
neighbor_dealers_rate_adj,0.075203,1.000000


In [815]:
firearm_homicide_large[["white_rate","neighbor_dealers_rate_adj"]].corr()

,white_rate,neighbor_dealers_rate_adj
white_rate,1.000000,0.020857
neighbor_dealers_rate_adj,0.020857,1.000000


### Compare gunbroker.com list to ATF FFL list

In [827]:
# Create geodataframe
gb_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

In [828]:
# Set to NAD 1983 Albers North America - https://epsg.io/102008
gb_gdf = gb_gdf.set_crs("EPSG:4269")

In [829]:
# Merge with county
gb_gdf_county = gb_gdf.sjoin(counties, how="left", predicate='intersects')
gb_gdf_county.drop('index_right', axis=1, inplace=True)

In [830]:
gb_gdf_county_merge = gb_gdf_county.groupby(["geoid"]).size().reset_index(name="dealers")

In [831]:
gb_counties_merge_full = counties[["geoid","state","county","county_area"]].merge(gb_gdf_county_merge, on="geoid", how="left")

In [832]:
gb_counties_merge_full["dealer_rate"] = gb_counties_merge_full["dealers"] / gb_counties_merge_full["county_area"]

In [833]:
gb_counties_merge_full.head()

,geoid,state,county,county_area,dealers,dealer_rate
0,02013,Alaska,Aleutians East,15009.939752,NaN,NaN
1,02016,Alaska,Aleutians West,14116.419570,1.0,0.000071
2,28107,Mississippi,Panola,705.131639,5.0,0.007091
3,28101,Mississippi,Newton,579.601941,4.0,0.006901
4,28027,Mississippi,Coahoma,583.152412,1.0,0.001715


In [837]:
ffl_counties = pd.read_csv("data/processed/dealers-list/2023/0123-ffl-list-county-summary.csv", dtype={"lic_regn":str,"lic_dist":str,"lic_seqn":str,"geoid":str})

In [836]:
ffl_counties.columns

Index(['statefp', 'state', 'countyfp', 'geoid', 'county', 'county_area',
       'county_population', 'county_white', 'county_black', 'county_asian',
       'county_latino', 'county_median_income', 'county_poverty', 'id',
       'business', 'pawn', 'sport', 'defense', 'big_box', 'small_business',
       'commercial', 'residential', 'all_density_land', 'all_density_pop',
       'business__pct', 'pawn__pct', 'sport_pct', 'defense_pct', 'big_box_pct',
       'small_business_pct', 'residential_pct', 'commercial_pct',
       'neighbor_id', 'neighbor_pawn', 'neighbor_sm_business', 'neighbor_area',
       'neighbor_population'],
      dtype='object')

In [838]:
compare = gb_counties_merge_full[["geoid","dealer_rate"]].merge(ffl_counties[["geoid","all_density_land"]], on="geoid", how="outer")

In [839]:
compare

,geoid,dealer_rate,all_density_land
0,02013,NaN,0.019987
1,02016,0.000071,0.028336
2,28107,0.007091,1.985445
3,28101,0.006901,1.552790
4,28027,0.001715,0.514445
...,...,...,...
3216,37077,0.016775,4.473452
3217,37111,0.024664,7.399191
3218,27021,0.003728,1.077053
3219,27057,0.004002,1.600705


In [843]:
compare["all_density_land"] = compare["all_density_land"] / 100

In [844]:
compare[["dealer_rate","all_density_land"]].corr()

,dealer_rate,all_density_land
dealer_rate,1.000000,0.845759
all_density_land,0.845759,1.000000


In [878]:
fig = alt.Chart(compare).mark_point().encode(
    x=alt.X('dealer_rate:Q', title="Gunbroker.com dealers per mile"),
    y=alt.Y('all_density_land:Q', title="All FFL dealers per mile"),
).properties(
    width=300,
    height=250
)
# making the regression line using transform_regression function and add with the scatter plot
final_plot = fig + fig.transform_regression('dealer_rate','all_density_land').mark_line()

In [879]:
final_plot

alt.LayerChart(...)